In [ ]:
from pathlib import Path as Path
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt


from h5py import File as hf
# from data import TNGData
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import DynamicEdgeConv, global_max_pool, knn_graph
import torch.optim as optim
from torch_geometric.loader import DataLoader
from tqdm import tqdm


from torch_geometric.data import Data

import os
import shutil
import numpy as np

import random

# Set seed
# SEED=42
# random.seed(SEED)
# torch.manual_seed(SEED)
# np.random.seed(SEED)

ROOT = "/home/group10/deephalo_gnn/TNG50-4-Dark/"

In [ ]:
sim = hf(ROOT + "simulation.hdf5", "r")

In [ ]:
# # Specify the path to the saved model
# model_path = "/home/group10/deephalo_gnn/ckpts/20240418-151905_pointnet_instance_seg_model.pth"

# # Load the model
# model = torch.load(model_path)
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load("/home/group10/deephalo_gnn/ckpts/20240418-151905_pointnet_instance_seg_model.pth"))

In [ ]:
data_test = "/home/group10/deephalo_gnn/New test"
files = os.listdir(data_test)
point_cloud_data = [(np.load(data_test+"/"+f)) for f in files if f.endswith(".npy")] # List of point cloud data, each element is a list of point coordinates

# Convert each point cloud data into a Data object
data_test_list = []
for point_cloud in point_cloud_data:
    # Create a Data object with the positions of points
    data_test = Data(pos=torch.tensor(point_cloud[:,:-1], dtype=torch.float), y = torch.eye(21)[torch.tensor(point_cloud[:,-1]+1, dtype=torch.long)])
    #
    data_test_list.append(data_test)

In [ ]:
model.eval()
# Loop over the test data
with torch.no_grad():
    for data in tqdm(test_loader, desc='Testing'):
        # Move data to the device
        data = data
        
        # Pass the data through the model
        outputs = model(data)
        
        # Get the predicted labels
        _, predicted_labels = torch.max(outputs, 1)
        _, ground_truth = torch.max(data.y, 1)
        # Store the predictions
        pos = data.pos.cpu().numpy()
        
        pos_list.append(pos)
        ground_truth_labels.append(ground_truth.cpu().numpy())
        predictions.append(predicted_labels.cpu().numpy())
        
ground_truth_labels = torch.argmax(ground_truth_labels, dim=1)
predictions = torch.argmax(predictions, dim=1)

# At this point, `predictions` is a list of numpy arrays with the predicted labels for each point cloud in the test set
# You can now compare these predictions to the actual labels to compute your test metrics

In [ ]:
hid = 101
halo_r = np.load("/home/group10/deephalo_gnn/Labeled subhalo matrices of haloes/101.npy") # Transposing here for ease-of-use later

fig = go.Figure(data=[
    go.Scatter3d(
        x=pos_list[3][:,0],
        y=pos_list[3][:,1],
        z=pos_list[3][:,2],
        mode='markers',
        marker=dict(
            size=1, # Larger than surrounding data-points
            color=ground_truth_labels[3],
            opacity=0.75,
            showscale=True,
        ))
])
fig.update_layout(
    
)

In [ ]:
hid = 101
halo_r = np.load("/home/group10/deephalo_gnn/Imbalance_Resampled_for_mulltilabel/train/101.npy") # Transposing here for ease-of-use later


fig = go.Figure(data=[
    go.Scatter3d(
        x=halo_r[:,0],
        y=halo_r[:,1],
        z=halo_r[:,2],
        mode='markers',
        marker=dict(
            size=1, # Larger than surrounding data-points
            color=halo_r[:,-1],
            opacity=0.75,
            showscale=True,
        ))
])
fig.update_layout(
    
)